In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
######### PRE - PROCESSING ################

import cv2
import os

# Define image size and output directory
img_width, img_height = 224, 224
output_dir = ('gdrive/My Drive/TEST')

# Define the pre-processing pipeline
def pre_process_image(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply thresholding
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Find and draw contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 10 and h > 10:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Resize the image
    img = cv2.resize(img, (img_width, img_height))

    return img

# Load the raw image dataset and pre-process each image
raw_data_dir = ('gdrive/My Drive/TEST/DATA')
for filename in os.listdir(raw_data_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(raw_data_dir, filename)
        pre_processed_image = pre_process_image(image_path)

        # Save the pre-processed image to the output directory
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, pre_processed_image)


In [ ]:
# import cv2
# import os

# # Define image size and output directory
# img_width, img_height = 224, 224
# output_dir = 'gdrive/My Drive/TEST/YOLO5'

# # Define the pre-processing pipeline
# def pre_process_image(image_path):
#     # Load the image
#     img = cv2.imread(image_path)

#     # Convert to grayscale
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#     # Apply Gaussian blur
#     blur = cv2.GaussianBlur(gray, (5, 5), 0)

#     # Apply thresholding
#     _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

#     # Find and draw contours
#     contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     for contour in contours:
#         x, y, w, h = cv2.boundingRect(contour)
#         if w > 10 and h > 10:
#             cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
#             cv2.putText(img, 'Car', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#     # Resize the image
#     img = cv2.resize(img, (img_width, img_height))

#     return img

# # Load the raw image dataset and pre-process each image
# raw_data_dir = 'gdrive/My Drive/TEST'
# for filename in os.listdir(raw_data_dir):
#     if filename.endswith('.jpg') or filename.endswith('.png'):
#         image_path = os.path.join(raw_data_dir, filename)
#         pre_processed_image = pre_process_image(image_path)

#         # Save the pre-processed image to the output directory
#         output_path = os.path.join(output_dir, filename)
#         cv2.imwrite(output_path, pre_processed_image)
# results.show()

In [ ]:
import cv2
import numpy as np
import os

# Load YOLOv3 weights and configuration
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load COCO class labels
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Generate random colors for each class label
np.random.seed(42)
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Path to directory containing input images
input_dir = 'gdrive/My Drive/TEST'

# Path to directory to save output images
output_dir = 'gdrive/My Drive/TEST/YOLO5'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over input images
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load image
        image = cv2.imread(os.path.join(input_dir, filename))

        # Get image dimensions
        height, width, _ = image.shape

        # Convert image to blob format for input to neural network
        blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), swapRB=True, crop=False)

        # Set input to neural network
        net.setInput(blob)

        # Run forward pass through network
        outs = net.forward(net.getUnconnectedOutLayersNames())

        # Initialize empty lists for boxes, confidences, and class IDs
        boxes = []
        confidences = []
        classIDs = []

        # Iterate over each detection output
        for out in outs:
            # Iterate over each detection in output
            for detection in out:
                # Extract class ID and confidence of current detection
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]

                # Filter out weak detections
                if confidence > 0.5:
                    # Calculate bounding box coordinates relative to image size
                    box = detection[0:4] * np.array([width, height, width, height])
                    (centerX, centerY, bwidth, bheight) = box.astype("int")
                    x = int(centerX - (bwidth / 2))
                    y = int(centerY - (bheight / 2))

                    # Add box, confidence, and class ID to respective lists
                    boxes.append([x, y, int(bwidth), int(bheight)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

        # Apply non-maxima suppression to remove overlapping boxes
        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # Iterate over indices after non-maxima suppression
        for i in indices:
            #i = i[0]
            box = boxes[i]
            x, y, w, h = box
            color = colors[classIDs[i]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = f"{classes[classIDs[i]]}: {confidences[i]:.2f}"
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Save output image
        cv2.imwrite(os.path.join(output_dir, filename), image)


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load the VGG16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the weights of the pre-trained layers
for layer in vgg16.layers:
    layer.trainable = False

# Add a new classifier on top of the pre-trained model
x = Flatten()(vgg16.output)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
predictions = Dense(1000, activation='softmax')(x)

# Define the new model
model = Model(inputs=vgg16.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


58889256/58889256 [==============================] - 0s 0us/step
